# Kaggle competition--Expedia

In [1]:
import pandas as pd

In [2]:
destinations = pd.read_csv("D:/Kaggle/expedia-data/destinations.csv")

In [3]:
sample_submission=pd.read_csv("D:/Kaggle/expedia-data/sample_submission.csv")

In [4]:
test = pd.read_csv("D:/Kaggle/expedia-data/test.csv")

In [ ]:
#train = pd.read_csv("D:/Kaggle/expedia-data/train.csv")

# Pandas提供了IO工具,可以将大文件分块读取

In [ ]:
reader = pd.read_csv('D:/Kaggle/expedia-data/train.csv', iterator=True)
try:
    train = reader.get_chunk(100000000)
except StopIteration:
    print "Iteration is stopped."

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2528243 entries, 0 to 2528242
Data columns (total 22 columns):
id                           int64
date_time                    object
site_name                    int64
posa_continent               int64
user_location_country        int64
user_location_region         int64
user_location_city           int64
orig_destination_distance    float64
user_id                      int64
is_mobile                    int64
is_package                   int64
channel                      int64
srch_ci                      object
srch_co                      object
srch_adults_cnt              int64
srch_children_cnt            int64
srch_rm_cnt                  int64
srch_destination_id          int64
srch_destination_type_id     int64
hotel_continent              int64
hotel_country                int64
hotel_market                 int64
dtypes: float64(1), int64(18), object(3)
memory usage: 443.6+ MB


In [11]:

train.shape

(2528243, 22)